In [132]:
import os
from pathlib import Path
import pandas as pd
import pydra
import nest_asyncio
nest_asyncio.apply()

In [133]:
tsv_path = "/Users/matthieu.joulot/Documents/QC/new/oasis_subjects_lin.tsv"
caps_path = "/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin"
#out_path = "/localdrive10TB/users/matthieu.joulot/QC_data/adni/qc_sandbox"
path_to_mni = "/Users/matthieu.joulot/Downloads/mni_icbm152_t1_tal_nlin_sym_09c.nii.gz"

In [150]:
def apply_BET(input_file):
    import subprocess
    output_path = str(Path(input_file).parent / Path(Path(input_file).stem).stem) + "_bet.nii.gz"
    print(output_path)
    cmd = f"bet {input_file} {output_path} -m"
    subprocess.run(cmd , shell=True)
    return str(output_path) #+ "nii.gz"

In [151]:
import os
import numpy as np
import nibabel as nib


def DICE(im1, im2, mode='bin'):
    if isinstance(im1, str) and os.path.isfile(im1):
        img1 = nib.load(im1).get_fdata()
    elif isinstance(im1, np.ndarray):
        img1 = im1
    else:
        raise ValueError("im1 can only be a path to a nifti or a np.ndarray")
    if isinstance(im2, str) and os.path.isfile(im2):
        img2 = nib.load(im2).get_fdata()
    elif isinstance(im2, np.ndarray):
        img2 = im2
    else:
        raise ValueError("im2 can only be a path to a nifti or a np.ndarray")

    if img1.shape != img2.shape:
        raise ValueError("Shape mismatch: im1 and im2 not the same shape")

    if mode == 'bin':
        img1 = img1 > 0.5
        img2 = img2 > 0.5
        return 2. * np.sum(np.logical_and(img1, img2)) / (img1.sum() + img2.sum())
    elif mode == 'contiuous':
        return 2 * np.sum(np.multiply(img1, img2)) / (img1.sum() + img2.sum())
    else:
        raise ValueError("Mode can only be continuous or bin")

In [172]:
def QC_t1_lin(caps_dir, data_tsv, path_to_mni):
    bids_df=pd.read_csv(data_tsv, sep="\t")
    bids_df.set_index(["participant_id", "session_id"], inplace=True)
    image_list = []
    bet_mni = apply_BET(path_to_mni)
    for subject, subject_df in bids_df.groupby(level=0):
        print("for 1")
        for (_, session) in subject_df.index.values:
            print("for 2")
            img_path = os.path.join(caps_dir, f"{subject}/{session}/t1_linear/{subject}_{session}_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz")
            print("img_path is: ", img_path)
            if Path(img_path).exists():
                print(img_path, " exists")
                bet_image_path = apply_BET(img_path)
                print("bet_image_path", bet_image_path)
                dice = DICE(bet_mni, bet_image_path)
                image_list.append((subject, session, dice))
    return image_list

In [167]:
## Readers

def read_images_t1_vol(caps_dir, data_tsv):
    bids_df=pd.read_csv(data_tsv, sep="\t")
    bids_df.set_index(["participant_id", "session_id"], inplace=True)
    image_list = []
    for subject, subject_df in bids_df.groupby(level=0):
        for (_, session) in subject_df.index.values:
            img_path_csf = os.path.join(caps_dir, f"subjects/{subject}/{session}/t1/spm/segmentation/native_space/{subject}_{session}_T1w_segm-csf_probability.nii.gz")
            img_path_gm = os.path.join(caps_dir, f"subjects/{subject}/{session}/t1/spm/segmentation/native_space/{subject}_{session}_T1w_segm-graymattter_probability.nii.gz")
            img_path_wm = os.path.join(caps_dir, f"subjects/{subject}/{session}/t1/spm/segmentation/native_space/{subject}_{session}_T1w_segm-whitematter_probability.nii.gz")
            if Path(img_path_csf).exists() and Path(img_path_csf).exists() and Path(img_path_csf).exists():
                image_list.append((img_path_csf, img_path_gm, img_path_wm))
    return image_list
            
def read_images_t1_lin(caps_dir, data_tsv):
    bids_df=pd.read_csv(data_tsv, sep="\t")
    bids_df.set_index(["participant_id", "session_id"], inplace=True)
    image_list = []
    for subject, subject_df in bids_df.groupby(level=0):
        for (_, session) in subject_df.index.values:
            img_path = os.path.join(caps_dir, f"subjects/{subject}/{session}/t1_linear/{subject}_{session}_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz")
            if Path(img_path).exists():
                image_list.append(img_path)
    return image_list

def read_images_pet_lin(caps_dir, data_tsv):
    bids_df=pd.read_csv(data_tsv, sep="\t")
    bids_df.set_index(["participant_id", "session_id"], inplace=True)
    image_list = []
    for subject, subject_df in bids_df.groupby(level=0):
        for (_, session) in subject_df.index.values:
            img_path = os.path.join(caps_dir, f"subjects/{subject}/{session}/pet_linear/{subject}_{session}_trc-fdg_pet_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_suvr-cerebellumPons2_pet.nii.gz")
            if Path(img_path).exists():
                image_list.append(img_path)
    return image_list

In [173]:
image_list_t1_lin = QC_t1_lin(caps_path, tsv_path, path_to_mni)

/Users/matthieu.joulot/Downloads/mni_icbm152_t1_tal_nlin_sym_09c_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10002/ses-M00/t1_linear/sub-OASIS10002_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10002/ses-M00/t1_linear/sub-OASIS10002_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10002/ses-M00/t1_linear/sub-OASIS10002_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10002/ses-M00/t1_linear/sub-OASIS10002_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10010/ses-M00/t1_linear/sub-OASIS10010_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/

bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10081/ses-M00/t1_linear/sub-OASIS10081_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10084/ses-M00/t1_linear/sub-OASIS10084_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10084/ses-M00/t1_linear/sub-OASIS10084_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10084/ses-M00/t1_linear/sub-OASIS10084_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10084/ses-M00/t1_linear/sub-OASIS10084_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10103/ses-M00/t1_linear/sub-O

bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10155/ses-M00/t1_linear/sub-OASIS10155_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10156/ses-M00/t1_linear/sub-OASIS10156_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10156/ses-M00/t1_linear/sub-OASIS10156_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10156/ses-M00/t1_linear/sub-OASIS10156_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10156/ses-M00/t1_linear/sub-OASIS10156_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10162/ses-M00/t1_linear/sub-O

bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10218/ses-M00/t1_linear/sub-OASIS10218_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10222/ses-M00/t1_linear/sub-OASIS10222_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10222/ses-M00/t1_linear/sub-OASIS10222_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10222/ses-M00/t1_linear/sub-OASIS10222_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10222/ses-M00/t1_linear/sub-OASIS10222_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10223/ses-M00/t1_linear/sub-O

bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10309/ses-M00/t1_linear/sub-OASIS10309_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10310/ses-M00/t1_linear/sub-OASIS10310_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10310/ses-M00/t1_linear/sub-OASIS10310_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10310/ses-M00/t1_linear/sub-OASIS10310_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10310/ses-M00/t1_linear/sub-OASIS10310_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10316/ses-M00/t1_linear/sub-O

bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10372/ses-M00/t1_linear/sub-OASIS10372_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10373/ses-M00/t1_linear/sub-OASIS10373_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10373/ses-M00/t1_linear/sub-OASIS10373_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10373/ses-M00/t1_linear/sub-OASIS10373_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10373/ses-M00/t1_linear/sub-OASIS10373_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10377/ses-M00/t1_linear/sub-O

bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10420/ses-M00/t1_linear/sub-OASIS10420_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10423/ses-M00/t1_linear/sub-OASIS10423_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10423/ses-M00/t1_linear/sub-OASIS10423_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz  exists
/Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10423/ses-M00/t1_linear/sub-OASIS10423_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
bet_image_path /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10423/ses-M00/t1_linear/sub-OASIS10423_ses-M00_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w_bet.nii.gz
for 1
for 2
img_path is:  /Users/matthieu.joulot/Documents/QC/new/oasis_caps_lin/sub-OASIS10430/ses-M00/t1_linear/sub-O

In [177]:
dice_df =pd.DataFrame(image_list_t1_lin).rename(columns={0:"participant_id", 1:"session_id", 2:"dice_probability"})

In [183]:
darq_df = pd.read_csv("/Users/matthieu.joulot/Documents/QC/new/darq_qc/qc_oasis.tsv", sep="\t").rename(columns={"pass_probability":"darq_probability", "pass": "darq_pass"})



In [184]:
dataframe = dice_df.merge(darq_df, how="inner", on=['participant_id', 'session_id'])

In [185]:
dataframe

,participant_id,session_id,dice_probability,darq_probability,darq_pass
0,sub-OASIS10002,ses-M00,0.892643,0.891636,True
1,sub-OASIS10010,ses-M00,0.811731,0.000141,False
2,sub-OASIS10011,ses-M00,0.930568,0.999974,True
3,sub-OASIS10037,ses-M00,0.923034,0.670437,True
4,sub-OASIS10042,ses-M00,0.893140,0.999505,True
...,...,...,...,...,...
73,sub-OASIS10430,ses-M00,0.863894,0.999974,True
74,sub-OASIS10432,ses-M00,0.939864,0.999980,True
75,sub-OASIS10435,ses-M00,0.936557,0.999535,True
76,sub-OASIS10439,ses-M00,0.930331,0.999980,True


In [186]:
pd.DataFrame(dataframe).to_csv("oasis_dice.tsv", sep="\t")